In [0]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
from pyspark.sql.types import (
    ArrayType,
    BinaryType,
    DoubleType,
    StringType,
    StructType,
    StructField,
    DateType,
)
from delta.tables import DeltaTable
from sedona.spark import *
from shapely import wkb
from shapely.ops import unary_union
import random
import time
import os
from dotenv import load_dotenv
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "helipads_bronze"
silver_table = "helipads_silver"

BASE_PATH = "/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering"
SUBDIR = {"image": "helipad_images", "mask": "helipad_labels"}

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {silver_table} (
    row_hash STRING,
    lokalids ARRAY<STRING>,
    geometry BINARY,
    bbox ARRAY<DOUBLE>,
    Polygons BINARY,
    adjusted_struct STRUCT<bbox: ARRAY<DOUBLE>, bbox_str: STRING>,
    Adjusted_bbox ARRAY<DOUBLE>,
    bbox_str STRING,
    image_path STRING,
    mask_path STRING,
    image_wms STRING,
    image_status STRING,
    mask_status STRING,
    type STRING,
    ingest_time TIMESTAMP,
    load_time TIMESTAMP,
    photo_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

In [0]:
def read_table_to_wkt():
    """
    Leser polygonene fra bronsetabellen og returnerer dem som en GeoDataFrame med WKT geometri.
    """
    df_bronze = spark.read.table(bronze_table).withColumn(
        "geometry", F.expr("ST_GeomFromWKB(geometry)")
    )
    df = (
        df_bronze.withColumn("_geom_srid", F.expr("ST_SetSRID(geometry, 4326)"))
        .withColumn(
            "geometry", F.expr("ST_Force2D(ST_Transform(_geom_srid, 'EPSG:25833'))")
        )
        .select("lokalid", "geometry", "ingest_time", "oppdateringsdato")
    )
    return df

In [0]:
def make_envelope(df: DataFrame, column_name: str) -> DataFrame:
    """
    Lager en minimal boks rundt polygonene.
    """
    return df.withColumn("envelope", F.expr(f"ST_Envelope({column_name})"))

In [0]:
def random_adjusted_bbox_centered(
    envelope: list, bbox_size: int = 128, max_offset: float = 20
) -> list:
    """
    Genererer en tilfeldig justert boks rundt polygonen med en maksimal avstand fra sentrum.
    """
    xmin, ymin, xmax, ymax = envelope
    poly_width = xmax - xmin
    poly_height = ymax - ymin

    if poly_width > bbox_size or poly_height > bbox_size:
        print("OBS: polygon er større enn bbox")
        max_offset = 0

    half_size = bbox_size / 2

    center_x = (xmin + xmax) / 2 + random.uniform(-max_offset, max_offset)
    center_y = (ymin + ymax) / 2 + random.uniform(-max_offset, max_offset)

    adjusted_xmin = center_x - half_size
    adjusted_xmax = center_x + half_size
    adjusted_ymin = center_y - half_size
    adjusted_ymax = center_y + half_size

    bbox = [adjusted_xmin, adjusted_ymin, adjusted_xmax, adjusted_ymax]
    bbox_str = ",".join(f"{v:.6f}" for v in bbox)
    return bbox, bbox_str

In [0]:
def envelope_to_bboxes(df: DataFrame, tile_width=128, tile_height=128) -> DataFrame:
    """
    Tar inn envelope og returnerer én bbox med gitt størrelse,
    sentrert på envelope.
    """
    df = (
        df.withColumn("xmin", F.expr("ST_XMin(envelope)"))
        .withColumn("ymin", F.expr("ST_YMin(envelope)"))
        .withColumn("xmax", F.expr("ST_XMax(envelope)"))
        .withColumn("ymax", F.expr("ST_YMax(envelope)"))
    )

    df = df.withColumn("x_center", (F.col("xmin") + F.col("xmax")) / 2.0).withColumn(
        "y_center", (F.col("ymin") + F.col("ymax")) / 2.0
    )

    df = df.withColumn("x", F.col("x_center") - F.lit(tile_width / 2.0)).withColumn(
        "y", F.col("y_center") - F.lit(tile_height / 2.0)
    )

    df = (
        df.withColumn(
            "tile",
            F.expr(
                f"ST_PolygonFromEnvelope(x, y, x + {tile_width}, y + {tile_height})"
            ),
        )
        .withColumn("bbox", F.expr(f"array(x, y, x + {tile_width}, y + {tile_height})"))
        .withColumn("bbox_str", F.concat_ws(",", F.col("bbox")))
    )

    drop_cols = [
        "xmin",
        "ymin",
        "xmax",
        "ymax",
        "x_center",
        "y_center",
        "x",
        "y",
        "tile",
    ]
    for c in drop_cols:
        if c in df.columns:
            df = df.drop(c)

    return df

In [0]:
def add_geometry_columns(df: DataFrame) -> DataFrame:
    """
    Legger til kolonner for boks og justert boks.
    """
    df = (
        df.withColumn(
            "Polygons", F.expr("ST_MakeEnvelope(bbox[0], bbox[1], bbox[2], bbox[3])")
        )
        .withColumn("adjusted_struct", adjusted_bbox_udf(F.col("bbox")))
        .withColumn("Adjusted_bbox", F.col("adjusted_struct.bbox"))
        .withColumn("bbox_str", F.col("adjusted_struct.bbox_str"))
        .drop("envelope")
    )
    return df

In [0]:
def generate_image_url(bbox_str: str) -> str:
    """
    Genererer en URL for image-bilde basert på bbox_str.
    """
    width, height = [512, 512]
    return (
        f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
        f"&service=WMS&request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
        f"BBox={bbox_str}&width={width}&height={height}"
    )


def image_file_exists(id: str) -> str:
    """
    Sjekker om bildet med gitt ID er lastet ned.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/helipad_images/image_{id}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"


def mask_file_exists(id: str) -> str:
    """
    Sjekker om masken med gitt ID er generert.
    """
    path = f"/Volumes/land_auto-gen-kart_dev/external_dev/static_data/DL_bildesegmentering/helipad_labels/mask_{id}.png"
    return "GENERATED" if os.path.exists(path) else "PENDING"

In [0]:
load_dotenv()
BRUKERID = os.getenv("GEONORGE_BRUKERID")
PASSORD = os.getenv("GEONORGE_PASSORD")


def get_token():
    """
    Henter token fra GeoNorge og returnerer det.
    """
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={BRUKERID}"
        f"&passord={PASSORD}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token


token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60


def refresh_token_if_needed():
    """
    Henter ny token om den gamle er utløpt.
    """
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
def get_fotodato(bbox: str, token: str, max_retries=10):
    """
    Henter fotodato for en bbox.
    """
    url = f"https://wms.geonorge.no/skwms1/wms.nib?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&CRS=EPSG:25833&BBOX={bbox}&WIDTH=512&HEIGHT=512&LAYERS=ortofoto&QUERY_LAYERS=ortofoto&INFO_FORMAT=text/html&I=256&J=256&TICKET={token}"

    table = None
    field_value = None
    for i in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            time.sleep(1.0)
            soup = BeautifulSoup(response.text, "html.parser")

            table = soup.find("table")
            break
        except Exception as e:
            wait = 2**i
            print(f"⚠️ Feil ved henting av fotodato ({e}), prøver igjen om {wait}s...")
            time.sleep(wait)

    if table:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2 and cells[0].text.strip() == "Fotodato":
                field_value = cells[1].text.strip()
                field_value = datetime.strptime(field_value, "%d.%m.%Y").date()
                return field_value

    return None

In [0]:
def add_ortofoto_date(df: DataFrame, token: str) -> DataFrame:
    """
    Legger til fotodato kolonnen til en DataFrame.
    """
    # Henter bare relevante kolonner
    sample_rows = df.select("row_hash", "bbox_str").collect()

    # Henter fotodato
    bbox_date_pairs = [
        (row["row_hash"], get_fotodato(row["bbox_str"].replace("_", ","), token))
        for row in sample_rows
    ]

    schema = StructType(
        [
            StructField("row_hash", StringType(), True),
            StructField("photo_time", DateType(), True),
        ]
    )

    bbox_date_df = spark.createDataFrame(bbox_date_pairs, schema)
    df_with_date = df.join(bbox_date_df, on="row_hash", how="left")

    return df_with_date

In [0]:
def build_overlap_graph(df: DataFrame) -> DataFrame:
    df_with_id = df.withColumn("row_id", F.monotonically_increasing_id())
    overlap_df = (
        df_with_id.alias("a")
        .join(
            df_with_id.alias("b"),
            F.expr(
                """
                ST_Intersects(
                    ST_MakeEnvelope(a.bbox[0], a.bbox[1], a.bbox[2], a.bbox[3]),
                    ST_MakeEnvelope(b.bbox[0], b.bbox[1], b.bbox[2], b.bbox[3])
                )
            """
            ),
        )
        .select(
            F.col("a.lokalid").alias("lokalid_a"),
            F.col("b.lokalid").alias("lokalid_b"),
        )
    )
    return overlap_df

In [0]:
def connected_components(edges: DataFrame) -> DataFrame:
    """
    Finner sammenhengende komponenter i en graf.
    """
    vertices = (
        edges.select("src")
        .union(edges.select("dst"))
        .distinct()
        .withColumn("component", F.col("src"))
    )
    changed = True
    while changed:
        propagated = (
            edges.alias("e")
            .join(vertices.alias("v"), F.col("e.src") == F.col("v.src"))
            .select(F.col("e.dst").alias("src"), F.col("v.component"))
        )
        new_vertices = (
            vertices.union(propagated)
            .groupBy("src")
            .agg(F.min("component").alias("component"))
        )
        joined = new_vertices.alias("n").join(vertices.alias("v"), "src")
        changed = (
            joined.filter(F.col("n.component") != F.col("v.component")).count() > 0
        )
        vertices = new_vertices
    return vertices

In [0]:
def merge_chain_geometries(chains_with_geom: DataFrame) -> DataFrame:
    """
    Slår sammen polygoner i en linje og returnerer det som en polygon.
    """
    merge_udf = F.udf(merge_polygons, BinaryType())
    chains_union = (
        chains_with_geom.groupBy("component")
        .agg(
            F.collect_list("geometry").alias("geom_list"),
            F.collect_list("lokalid").alias("lokalids"),
        )
        .withColumn("merged_wkb", merge_udf("geom_list"))
        .withColumn("merged_geom", F.expr("ST_GeomFromWKB(merged_wkb)"))
        .withColumn("type", F.lit("chain"))
    )
    chains_union = make_envelope(chains_union, "merged_geom")
    chains_union = envelope_to_bboxes(chains_union)
    return chains_union

In [0]:
def enrich_output(df: DataFrame, token: str) -> DataFrame:
    """
    Legger til kolonner med info om bilder, masker og tid.
    """
    df = add_geometry_columns(df)
    for dt in ["image", "mask"]:
        sub = SUBDIR[dt]
        df = df.withColumn(
            f"{dt}_path",
            F.concat(
                F.lit(f"{BASE_PATH}/{sub}/{dt}_"), F.col("row_hash"), F.lit(".png")
            ),
        )
    df = (
        df.withColumn("image_wms", generate_image_url_udf("bbox_str"))
        .withColumn("image_status", image_file_exists_udf("row_hash"))
        .withColumn("mask_status", mask_file_exists_udf("row_hash"))
        .withColumn("load_time", F.current_timestamp())
    )
    df = add_ortofoto_date(df, token)
    return df

In [0]:
def merge_polygons(polys):
    """
    Slår sammen polygoner fra en liste til en enkelt polygon.
    """
    shapes = [p for p in polys if p is not None]
    if not shapes:
        return None
    merged = unary_union(shapes)
    return merged.wkb

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og opdaterer dersom row_hash allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(silver_table)
    else:
        from delta.tables import DeltaTable

        delta_tbl = DeltaTable.forName(spark, silver_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.row_hash = source.row_hash"
        ).whenMatchedUpdate(
            condition="target.load_time < source.load_time OR target.image_path IS NULL",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

In [0]:
adjusted_bbox_schema = StructType(
    [
        StructField("bbox", ArrayType(DoubleType())),
        StructField("bbox_str", StringType()),
    ]
)
adjusted_bbox_udf = F.udf(
    lambda envelope: random_adjusted_bbox_centered(envelope), adjusted_bbox_schema
)
generate_image_url_udf = F.udf(generate_image_url, StringType())
image_file_exists_udf = F.udf(image_file_exists, StringType())
mask_file_exists_udf = F.udf(mask_file_exists, StringType())
merge_udf = F.udf(merge_polygons, BinaryType())

df = read_table_to_wkt()
df = make_envelope(df, "geometry")
df = envelope_to_bboxes(df)

edges = (
    build_overlap_graph(df)
    .select("lokalid_a", "lokalid_b")
    .withColumnRenamed("lokalid_a", "src")
    .withColumnRenamed("lokalid_b", "dst")
)
vertices = connected_components(edges)

chains = vertices.groupBy("component").agg(F.collect_list("src").alias("lokalids"))
chains_filtered = chains.filter(F.size("lokalids") > 1)
chains_exploded = chains_filtered.withColumn("lokalid", F.explode("lokalids"))
chains_with_geom = chains_exploded.join(df, "lokalid", "inner").select(
    "component", "lokalid", "geometry"
)

chains_union = merge_chain_geometries(chains_with_geom)

chain_ids = chains_filtered.select(F.explode("lokalids").alias("lokalid"))
singletons = (
    df.join(chain_ids, "lokalid", "left_anti")
    .select("lokalid", "geometry", "envelope", "bbox")
    .withColumn("merged_geom", F.col("geometry"))
    .withColumn("lokalids", F.array("lokalid"))
    .withColumn("type", F.lit("singleton"))
)

new_df = chains_union.select(
    "component", "lokalids", "merged_geom", "envelope", "bbox", "type"
).unionByName(
    singletons.select(
        F.col("lokalid").alias("component"),
        "lokalids",
        "merged_geom",
        "envelope",
        "bbox",
        "type",
    )
)

df = (
    new_df.withColumn(
        "row_hash", F.sha2(F.concat_ws("||", F.col("lokalids"), F.col("bbox")), 256)
    )
    .withColumnRenamed("merged_geom", "geometry")
    .drop("component")
)
df = enrich_output(df, token)

df = df.withColumn("geometry", F.expr("ST_AsBinary(geometry)"))
df = df.withColumn("Polygons", F.expr("ST_AsBinary(Polygons)"))
write_delta_table(df)